In [23]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance
import albumentations as A
import albumentations.pytorch
from tqdm.notebook import tqdm

import cv2
import re
import time

import sys
sys.path.append('../utils')
sys.path.append('../')
from visualize import *
from xml_parser import *
from transformations import *
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
# from retinanet import retina
from retinanet.retina import *
from retinanet.augment import *
from retinanet.dataloader import *
from retinanet.anchors import Anchors

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler

In [28]:
class Anchors(nn.Module):
    def __init__(self, pyramid_levels=None, strides=None, sizes=None, ratios=None, scales=None):
        super(Anchors, self).__init__()

        self.pyramid_levels = pyramid_levels
        self.strides = strides
        self.sizes = sizes
        self.ratios = ratios
        self.scales = scales

        if pyramid_levels is None:
            self.pyramid_levels = [3, 4, 5, 6, 7]
        if strides is None:
            self.strides = [2 ** x for x in self.pyramid_levels]
        if sizes is None:
            self.sizes = [2 ** (x + 2) for x in self.pyramid_levels]
        if ratios is None:
            self.ratios = np.array([0.5, 1, 2])
        if scales is None:
            self.scales = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])
        print(self.pyramid_levels )

    def forward(self, image):
        
        image_shape = image.shape[2:]
        image_shape = np.array(image_shape)
        image_shapes = [(image_shape + 2 ** x - 1) // (2 ** x) for x in self.pyramid_levels]

        # compute anchors over all pyramid levels
        all_anchors = np.zeros((0, 4)).astype(np.float32)

        for idx, p in enumerate(self.pyramid_levels):
            anchors         = generate_anchors(base_size=self.sizes[idx], ratios=self.ratios, scales=self.scales)
            shifted_anchors = shift(image_shapes[idx], self.strides[idx], anchors)
            all_anchors     = np.append(all_anchors, shifted_anchors, axis=0)

        all_anchors = np.expand_dims(all_anchors, axis=0)
#         print((all_anchors))

        if torch.cuda.is_available():
            return torch.from_numpy(all_anchors.astype(np.float32)).cuda()
        else:
            return torch.from_numpy(all_anchors.astype(np.float32))

def generate_anchors(base_size=16, ratios=None, scales=None):
    """
    Generate anchor (reference) windows by enumerating aspect ratios X
    scales w.r.t. a reference window.
    """

    if ratios is None:
        ratios = np.array([0.5, 1, 2])

    if scales is None:
        scales = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

    num_anchors = len(ratios) * len(scales)

    # initialize output anchors
    anchors = np.zeros((num_anchors, 4))

    # scale base_size
    anchors[:, 2:] = base_size * np.tile(scales, (2, len(ratios))).T

    # compute areas of anchors
    areas = anchors[:, 2] * anchors[:, 3]

    # correct for ratios
    anchors[:, 2] = np.sqrt(areas / np.repeat(ratios, len(scales)))
    anchors[:, 3] = anchors[:, 2] * np.repeat(ratios, len(scales))

    # transform from (x_ctr, y_ctr, w, h) -> (x1, y1, x2, y2)
    anchors[:, 0::2] -= np.tile(anchors[:, 2] * 0.5, (2, 1)).T
    anchors[:, 1::2] -= np.tile(anchors[:, 3] * 0.5, (2, 1)).T

    return anchors

def compute_shape(image_shape, pyramid_levels):
    """Compute shapes based on pyramid levels.

    :param image_shape:
    :param pyramid_levels:
    :return:
    """
    image_shape = np.array(image_shape[:2])
    image_shapes = [(image_shape + 2 ** x - 1) // (2 ** x) for x in pyramid_levels]
    return image_shapes


def anchors_for_shape(
    image_shape,
    pyramid_levels=None,
    ratios=None,
    scales=None,
    strides=None,
    sizes=None,
    shapes_callback=None,
):

    image_shapes = compute_shape(image_shape, pyramid_levels)

    # compute anchors over all pyramid levels
    all_anchors = np.zeros((0, 4))
    for idx, p in enumerate(pyramid_levels):
        anchors         = generate_anchors(base_size=sizes[idx], ratios=ratios, scales=scales)
        shifted_anchors = shift(image_shapes[idx], strides[idx], anchors)
        all_anchors     = np.append(all_anchors, shifted_anchors, axis=0)

    return all_anchors


def shift(shape, stride, anchors):
    shift_x = (np.arange(0, shape[1]) + 0.5) * stride
    shift_y = (np.arange(0, shape[0]) + 0.5) * stride

    shift_x, shift_y = np.meshgrid(shift_x, shift_y)

    shifts = np.vstack((
        shift_x.ravel(), shift_y.ravel(),
        shift_x.ravel(), shift_y.ravel()
    )).transpose()

    # add A anchors (1, A, 4) to
    # cell K shifts (K, 1, 4) to get
    # shift anchors (K, A, 4)
    # reshape to (K*A, 4) shifted anchors
    A = anchors.shape[0]
    K = shifts.shape[0]
    all_anchors = (anchors.reshape((1, A, 4)) + shifts.reshape((1, K, 4)).transpose((1, 0, 2)))
    all_anchors = all_anchors.reshape((K * A, 4))

    return all_anchors        

In [32]:
ratios=np.array([1])
scales=np.array([1])
# anchors = Anchors(pyramid_levels=[3, 4, 5], strides=None, sizes=None, ratios=ratios, scales=scales)
anchors = Anchors(pyramid_levels=None, strides=None, sizes=None, ratios=None, scales=None)

[3, 4, 5, 6, 7]


In [33]:
image = torch.randn([2, 3, 640, 640])
image.shape

torch.Size([2, 3, 640, 640])

In [34]:
ans = anchors(image)

[[[ -18.627417     -7.3137085    26.627417     15.3137085 ]
  [ -24.50875898  -10.25437949   32.50875898   18.25437949]
  [ -31.91878555  -13.95939277   39.91878555   21.95939277]
  ...
  [ 394.98066402  213.96132803  757.01933598  938.03867197]
  [ 347.92992816  119.85985631  804.07007184 1032.14014369]
  [ 288.64971563    1.29943127  863.35028437 1150.70056873]]]


In [22]:
ans[0][0:10]

tensor([[-12., -12.,  20.,  20.],
        [ -4., -12.,  28.,  20.],
        [  4., -12.,  36.,  20.],
        [ 12., -12.,  44.,  20.],
        [ 20., -12.,  52.,  20.],
        [ 28., -12.,  60.,  20.],
        [ 36., -12.,  68.,  20.],
        [ 44., -12.,  76.,  20.],
        [ 52., -12.,  84.,  20.],
        [ 60., -12.,  92.,  20.]], device='cuda:0')

In [36]:
ratios = np.array([0.5, 1, 2])
ratios

array([0.5, 1. , 2. ])

In [37]:
scales = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])
scales

array([1.        , 1.25992105, 1.58740105])

In [39]:
num_anchors = len(ratios) * len(scales)
num_anchors

9

In [40]:
# initialize output anchors
anchors = np.zeros((num_anchors, 4))
anchors

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [42]:
# scale base_size
base_size = 16
anchors[:, 2:] = base_size * np.tile(scales, (2, len(ratios))).T
anchors

array([[ 0.        ,  0.        , 16.        , 16.        ],
       [ 0.        ,  0.        , 20.1587368 , 20.1587368 ],
       [ 0.        ,  0.        , 25.39841683, 25.39841683],
       [ 0.        ,  0.        , 16.        , 16.        ],
       [ 0.        ,  0.        , 20.1587368 , 20.1587368 ],
       [ 0.        ,  0.        , 25.39841683, 25.39841683],
       [ 0.        ,  0.        , 16.        , 16.        ],
       [ 0.        ,  0.        , 20.1587368 , 20.1587368 ],
       [ 0.        ,  0.        , 25.39841683, 25.39841683]])

In [43]:
# compute areas of anchors
areas = anchors[:, 2] * anchors[:, 3]
areas

array([256.        , 406.3746693 , 645.07957755, 256.        ,
       406.3746693 , 645.07957755, 256.        , 406.3746693 ,
       645.07957755])

In [45]:
# correct for ratios
anchors[:, 2] = np.sqrt(areas / np.repeat(ratios, len(scales)))
anchors[:, 3] = anchors[:, 2] * np.repeat(ratios, len(scales))
anchors

array([[ 0.        ,  0.        , 22.627417  , 11.3137085 ],
       [ 0.        ,  0.        , 28.50875898, 14.25437949],
       [ 0.        ,  0.        , 35.91878555, 17.95939277],
       [ 0.        ,  0.        , 16.        , 16.        ],
       [ 0.        ,  0.        , 20.1587368 , 20.1587368 ],
       [ 0.        ,  0.        , 25.39841683, 25.39841683],
       [ 0.        ,  0.        , 11.3137085 , 22.627417  ],
       [ 0.        ,  0.        , 14.25437949, 28.50875898],
       [ 0.        ,  0.        , 17.95939277, 35.91878555]])

In [46]:

# transform from (x_ctr, y_ctr, w, h) -> (x1, y1, x2, y2)
anchors[:, 0::2] -= np.tile(anchors[:, 2] * 0.5, (2, 1)).T
anchors[:, 1::2] -= np.tile(anchors[:, 3] * 0.5, (2, 1)).T
anchors

array([[-11.3137085 ,  -5.65685425,  11.3137085 ,   5.65685425],
       [-14.25437949,  -7.12718975,  14.25437949,   7.12718975],
       [-17.95939277,  -8.97969639,  17.95939277,   8.97969639],
       [ -8.        ,  -8.        ,   8.        ,   8.        ],
       [-10.0793684 , -10.0793684 ,  10.0793684 ,  10.0793684 ],
       [-12.69920842, -12.69920842,  12.69920842,  12.69920842],
       [ -5.65685425, -11.3137085 ,   5.65685425,  11.3137085 ],
       [ -7.12718975, -14.25437949,   7.12718975,  14.25437949],
       [ -8.97969639, -17.95939277,   8.97969639,  17.95939277]])

In [ ]:
# initialize output anchors

anchors = np.zeros((num_anchors, 4))

# scale base_size
anchors[:, 2:] = base_size * np.tile(scales, (2, len(ratios))).T

# compute areas of anchors
areas = anchors[:, 2] * anchors[:, 3]

# correct for ratios
anchors[:, 2] = np.sqrt(areas / np.repeat(ratios, len(scales)))
anchors[:, 3] = anchors[:, 2] * np.repeat(ratios, len(scales))

# transform from (x_ctr, y_ctr, w, h) -> (x1, y1, x2, y2)
anchors[:, 0::2] -= np.tile(anchors[:, 2] * 0.5, (2, 1)).T
anchors[:, 1::2] -= np.tile(anchors[:, 3] * 0.5, (2, 1)).T